In [ ]:
from pandas_datareader import data as web
import pandas as pd
import numpy as np
from datetime import datetime as dt
import calendar
import io
import requests
import math
from scipy.optimize import minimize

In [ ]:
start_date = ['2017-01-01', '2018-01-01', '2019-01-01', '2020-01-01']
end_date = ['2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01'] 


Tickers = ['MSFT', 'AAPL', 'AMZN', 'TSLA', 'GOOGL', 'GOOG', 'FB', 'JPM', 'HD', 'JNJ', 'UNH', 'PG', 'BAC', 'V', 'ADBE', 'PFE', 'NFLX', 'MA', 'CRM', 'XOM', 'TMO', 'COST', 'CMCSA', 'CSCO', 'AVGO', 'ACN', 'ABT', 'PEP', 'PYPL', 'NKE']
print(Tickers)

['MSFT', 'AAPL', 'AMZN', 'TSLA', 'GOOGL', 'GOOG', 'FB', 'JPM', 'HD', 'JNJ', 'UNH', 'PG', 'BAC', 'V', 'ADBE', 'PFE', 'NFLX', 'MA', 'CRM', 'XOM', 'TMO', 'COST', 'CMCSA', 'CSCO', 'AVGO', 'ACN', 'ABT', 'PEP', 'PYPL', 'NKE']


In [ ]:
yBase = 'https://query1.finance.yahoo.com/v7/finance/download/'
yHeaders = {     'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0',     'Accept': 'text/csv;charset=utf-8'     }

def getYahooDf(ticker, startDate, endDate=None): # dates in ISO format
    start = dt.fromisoformat(startDate) # To datetime.datetime object
    fromDate = calendar.timegm(start.utctimetuple()) # To Unix timestamp format used by Yahoo
    if endDate is None:
        end=dt.now()
    else:
        end = dt.fromisoformat(endDate)
    toDate = calendar.timegm(end.utctimetuple())
    params = { 
        'period1': str(fromDate),
        'period2': str(toDate),
        'interval': '1d',
        'events': 'history',
        'includeAdjustedClose': 'true'
    }
    response = requests.request("GET", yBase + ticker, headers=yHeaders, params=params)
    if response.status_code < 200 or response.status_code > 299:
        print("here")
        return None
    else:
        csv = io.StringIO(response.text)
        df = pd.read_csv(csv, index_col='Date')
        return df


In [ ]:
df_T = []
for T in range(len(start_date)):
  df = pd.DataFrame()
  for stock in Tickers:
    df[stock] = getYahooDf(stock,start_date[T],end_date[T])['Adj Close']
  df_T.append(df)
print(df_T)

In [ ]:
log_returns_T = []

for df in df_T:
  log_returns = np.log(df/df.shift(1))
  log_returns =  log_returns.dropna()
  log_returns_T.append(log_returns)

print(log_returns_T)

In [ ]:
Cov_matrix_T = []

for log_returns in log_returns_T:
  Cov_matrix = log_returns.cov()
  Cov_matrix_T.append(Cov_matrix)

print(Cov_matrix_T)

In [ ]:
mean_returns_T = []

for log_returns in log_returns_T:
  mean_returns = log_returns.mean()
  mean_returns_T.append(mean_returns)

print(mean_returns_T)

In [ ]:
T = 4
N = 20
k = 8
X = log_returns_T
Q = Cov_matrix_T
EX = mean_returns_T
E = 0.0012

# Stochastic Gradient descent with layers

In [ ]:
def SNNr(t, E, k):

  def softmax(S):  
    S_exp = np.exp(S)
    sums = S_exp.sum(axis = 0)
    pi = S_exp/sums
    return pi

  def onehot(l):
    z = pd.DataFrame(np.zeros(l.shape),columns = X[t].columns)
    for i in range(l.shape[0]):
        z.iloc[i,:] = (l.iloc[i,:] == l.iloc[i,:].max()).astype(float)
    return z

  def minlossindex(l):
    index = np.where(l == np.amin(l))
    return index[0][0]

  ##main funcction

  def grad_update(S,w_tilda,alpha,k):

    for i in range(X[t].shape[0]):
        
        ##forward propagation
        pi = softmax(S)
        u = np.random.uniform(low = 0.0, high = 1.00, size = pi.shape)
        g = -np.log(-np.log(u))
        g = pd.DataFrame(g,columns = X[t].columns)
        gumbel_pi = g + np.log(pi)
        tz = onehot(gumbel_pi)
        z = np.array(tz.sum(axis = 0),ndmin = 2)
        w_prime = np.exp(w_tilda)
        w_bar = np.multiply(w_prime, z)
        w = w_bar/w_bar.sum()
        #print(pi)
        #print(u)
        #print(g)
        #print(gumbel_pi)
        #print(z)
        #print(w_prime)
        #print(w_bar)
        #print(w)
        risk = np.dot(np.dot(z,Q[t]),z.T)[0][0]
        Loss = np.square(np.dot(np.array(X[t].iloc[i,:],ndmin = 2),w.T)[0][0] - E) + risk
        loss_list.append(Loss)
        S_list.append(np.array(S))
        w_tilda_list.append(w_tilda)

        #print(Loss)

        ##gradient calc and descent
        a = np.dot(np.array(X[t].iloc[i,:],ndmin = 2),w.T)[0][0] - E
        b = (1/np.square(w_bar.sum()))
        c = np.multiply(z,w_prime)
        d = np.multiply(w_prime,S - pi)
        
        ##
        first = np.dot(w,Q[t].T+Q[t])
        second = np.dot(w_prime,(S-pi).T)
        third = np.dot(second.T,first)
        risk_grad = third*b
        
        e = np.multiply(first,w_prime)
        
        dw_tilda = a*b*c + b*e
        dS  = a*b*d + risk_grad

        ##update parameter
        w_tilda = w_tilda - (alpha)*(dw_tilda)
        S = S - (alpha)*(dS)

  alphas = [0.1,0.3,1,3,10]
  S = np.random.randn(k,X[t].shape[1])                         
  S = pd.DataFrame(S,columns= X[t].columns)
  w_tilda = np.random.randn(1,X[t].shape[1])

  alpha_losses_list = []        #holds min losses for each alpha
  alpha_S_list = []      #holds min S corresponding to index of min losses
  alpha_w_tilda_list = []       #holds min w_tilda corresponding to index of min losses

  for i in alphas:
    loss_list = []      #holds all losses for each alpha(per iteration)
    S_list = []         #holds all S for each alpha(per iteration)
    w_tilda_list = []    #holds all w_tilda for each alpha(per iteration)
    
    #run model
    grad_update(S,w_tilda,i,k)     
    
    #some updation of lists for optimization
    loss_list = np.array(loss_list,ndmin = 2).T
    w_tilda_list = np.array(w_tilda_list)
    S_list = np.array(S_list)
    ez = minlossindex(loss_list)
    loss_min = loss_list[ez]
    S_min = S_list[ez]
    w_tilda_min = w_tilda_list[ez]
    #print(i)
    alpha_losses_list.append(loss_min[0])
    alpha_S_list.append(S_min)
    alpha_w_tilda_list.append(w_tilda_min)

  ##changing types
  alpha_losses_list = np.array(alpha_losses_list,ndmin= 2)
  alpha_S_list = np.array(alpha_S_list)
  alpha_w_tilda_list = np.array(alpha_w_tilda_list)

  gg = minlossindex(alpha_losses_list)      #index minimum of all the losses in land(of all alphas)
  alpha_losses_list_min = alpha_losses_list[0][gg]           #S corresponding to min loss
  alpha_S_list_min = alpha_S_list[gg]          #w_tilda corresponding to min loss
  alpha_w_tilda_list_min = alpha_w_tilda_list[gg] 

  ## calculate w from feed forwarding
  pi = softmax(alpha_S_list_min)
  pi = pd.DataFrame(np.array(pi),columns = X[t].columns)
  tz = onehot(pi)
  z = np.array(tz.sum(axis = 0),ndmin = 2)
  w_prime = np.exp(alpha_w_tilda_list_min)
  w_bar = np.multiply(w_prime, z)
  w = w_bar/w_bar.sum()
  #print(w)

  return z

In [ ]:
Z = SNNr(0,E,k)
Z

In [ ]:
# Single-Period MV Model

def optimal_portfolio(N, Q, EX, Er,T):

  def risk(w, Q_t):
    return np.dot(w.T,np.dot(Q_t,w))

  def checkExpectedReturn(w, Er_min, Ex_t):
    res = np.dot(w.T,Ex_t) - Er_min
    return res

  def checkSum(w):
    sum = np.sum(w)
    return sum - 1

  w0 = np.array([1]*N)
  w0 = w0/w0.sum()

  weights = [w0 for i1 in range(T)]

  Ex_return = [Er for j1 in range(T)]

  risk_t = [0 for k1 in range(T)]

  for t in range(T):

    Z = SNNr(t, Ex_return[t], k)

    constraints = [{'type': 'ineq', 'fun': checkExpectedReturn, 'args': (Ex_return[t], EX[t],)},
               {'type': 'eq', 'fun': checkSum}]

    bounds = [(0,1) for i in range(N)]
    for i in range(N):
      if Z[0][i]!=0:
        bounds[i] = (0,1)

      if Z[0][i]==0:
        bounds[i] = (0,0)

    opt = minimize(risk, w0, method='SLSQP', args=(Q[t],), bounds=bounds, constraints=constraints)  # Finding optimal weights for each period

    weights[t] = opt.x

    risk_t[t] = risk(opt.x, Q[t])


  return weights, Ex_return, risk_t


[opt_weights1, Er_t1, r_t1] = optimal_portfolio(N, Q, EX, E, T)
print(opt_weights1)
print(Er_t1)
print(r_t1)

In [ ]:
w1 = opt_weights1
risk_t1 = []
Expected_t1 = []
for t in range (T):
  risk = np.dot(w1[t].T, np.dot(Q[t],w1[t]))
  risk_t1.append(risk)
  Et = np.dot(w1[t].T, EX[t])
  Expected_t1.append(Et)

print(risk_t1)
print(Expected_t1)

In [ ]:
# Multi-Period MV Model using prospect function

def optimal_portfolio_multi(N, Q, EX, Er, T):

  w0 = np.array([1]*N)
  w0 = w0/w0.sum()

  weights = [w0 for i2 in range(T)]

  Ex_return = [Er for j2 in range(T)]

  risk_t = [0 for k2 in range(T)]

  def risk(w, Q_t):
    return np.dot(w.T,np.dot(Q_t,w))

  def checkExpectedReturn(w, Er_min, Ex_t):
    res = np.dot(w.T,Ex_t) - Er_min
    return res

  def checkSum(w):
    sum = np.sum(w)
    return sum - 1

  Rr = 0.00022    # Expected risk level of investor
  
  def prospectFunc(w, Q_t):  # Calculating the asymmetric investor sentiment
    diff = np.dot(w.T, np.dot(Q_t,w)) - Rr
    a = 0.88
    b = 0.88
    Lambda = 1   # Values of Lambda and Theta depend on the investor sentiments
    Theta = 2
    if diff<0:
      return Lambda*(-1*diff)**a

    else:
      return -1*Theta*(diff)**b

  for t in range(T):

    Z = SNNr(t, Ex_return[t], k)

    constraints = [{'type': 'ineq', 'fun': checkExpectedReturn, 'args': (Ex_return[t], EX[t],)},
               {'type': 'eq', 'fun': checkSum}]

    bounds = [(0,1) for i in range(N)]
    for i in range(N):
      if Z[0][i]!=0:
        bounds[i] = (0,1)

      if Z[0][i]==0:
        bounds[i] = (0,0)

    opt = minimize(risk, weights[t], method='SLSQP', args=(Q[t],), bounds=bounds, constraints=constraints)  # Finding optimal weights for each period

    weights[t] = opt.x

    risk_t[t] = risk(opt.x, Q[t])

    if t<T-1: 
      Ex_return[t+1] = Ex_return[t]*(1 + prospectFunc(weights[t], Q[t]))   # Updating the expected return level for next period using the asymmetric investor sentiment

  return weights, Ex_return, risk_t


[opt_weights, Er_t, r_t] = optimal_portfolio_multi(N, Q, EX, E, T)
print(opt_weights)
print(Er_t)
print(r_t)

In [ ]:
w = opt_weights
risk_t = []
Expected_t = []
for t in range (T):
  risk = np.dot(w[t].T, np.dot(Q[t],w[t]))
  risk_t.append(risk)
  Et = np.dot(w[t].T, EX[t])
  Expected_t.append(Et)

print(risk_t)
print(Expected_t)